In [2]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt


In [3]:
# Load in the data

cities = ['barcelona', 'madrid', 'malaga', 'mallorca', 'menorca', 'sevilla', 'valencia']
dir_data = "../airbnb_data/"
calendar = pd.DataFrame()
listings = pd.DataFrame()
neighbourhoods = pd.DataFrame()
reviews = pd.DataFrame()
# dtype={'price': np.string_, 'availability': np.string_, 'adjusted_price': np.string_, 'minimum_nights': np.float64, 'maximum_nights': np.float64}

for city in cities:
    temp_calendar = pd.read_csv(f'{dir_data}{city}/calendar.csv', parse_dates=['date'], low_memory=False)
    temp_calendar['city'] = cities.index(city) + 1
    calendar = pd.concat([temp_calendar, calendar])

    temp_listings = pd.read_csv(f'{dir_data}{city}/listings.csv')
    temp_listings['city'] = cities.index(city) + 1
    listings = pd.concat([temp_listings, listings])

    temp_neighbourhoods = pd.read_csv(f'{dir_data}{city}/neighbourhoods.csv')
    temp_neighbourhoods['city'] = cities.index(city) + 1
    neighbourhoods = pd.concat([temp_neighbourhoods, neighbourhoods])

    temp_reviews = pd.read_csv(f'{dir_data}{city}/reviews.csv')
    temp_reviews['city'] = cities.index(city) + 1
    reviews = pd.concat([temp_reviews, reviews])

    # Added the compressed files 

    temp_reviews_gz = pd.read_csv(f'{dir_data}{city}/reviews_gz.csv')
    temp_reviews_gz['city'] = cities.index(city) + 1
    reviews_gz = pd.concat([temp_reviews, reviews])

    temp_listings_gz = pd.read_csv(f'{dir_data}{city}/listings_gz.csv')
    temp_listings_gz['city'] = cities.index(city) + 1
    listings_gz = pd.concat([temp_reviews, reviews])

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,city
0,48154,2024-06-22,f,$75.00,NaN,3.0,25.0,7
1,48154,2024-06-23,f,$75.00,NaN,3.0,25.0,7
2,48154,2024-06-24,f,$75.00,NaN,5.0,22.0,7
3,48154,2024-06-25,f,$75.00,NaN,5.0,22.0,7
4,48154,2024-06-26,f,$75.00,NaN,5.0,22.0,7
...,...,...,...,...,...,...,...,...
6907228,473459,2025-06-10,t,$345.00,NaN,4.0,365.0,1
6907229,473459,2025-06-11,t,$345.00,NaN,4.0,365.0,1
6907230,473459,2025-06-12,t,$345.00,NaN,4.0,365.0,1
6907231,473459,2025-06-13,t,$345.00,NaN,4.0,365.0,1


In [29]:
# Create dataframes from SQL tables
# Calendar table
proc_calendar = pd.DataFrame(columns=[
    'listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'
])

# Property type table
proc_property_type = pd.DataFrame(columns=['id', 'type'])

# Listings table
proc_listings = pd.DataFrame(columns=[
    'id', 'name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'city', 'latitude', 'longitude',
    'room_type', 'property_type', 'price', 'minimum_nights', 'maximum_nights', 'license'
])

# Listings description table
proc_listings_description = pd.DataFrame(columns=[
    'listing_id', 'description', 'picture_url', 'accommodates', 'bathrooms', 'bathrooms_text',
    'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'
])

# Listings amenities table
proc_listings_amenities = pd.DataFrame(columns=['listing_id', 'amenity'])

# Listings availability table
proc_listings_availability = pd.DataFrame(columns=[
    'listing_id', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
    'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
    'instant_bookable'
])

# Listings reviews table
proc_listings_reviews = pd.DataFrame(columns=[
    'listing_id', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'reviews_per_month'
])

# Neighborhood groups table
proc_neighborhood_groups = pd.DataFrame(columns=['id', 'name', 'neighbourhood_group_cleansed'])

# Neighborhoods table
proc_neighborhoods = pd.DataFrame(columns=[
    'id', 'name', 'neighborhood_cleansed', 'neighborhood_group', 'overview', 'city'
])

# Countries table
proc_countries = pd.DataFrame(columns=['iso_3166_1', 'name'])

# Cities table
proc_cities = pd.DataFrame(columns=['id', 'name', 'country_code'])

# Hosts table
proc_hosts = pd.DataFrame(columns=[
    'id', 'url', 'name', 'host_since', 'location', 'host_about', 'response_time', 'response_rate',
    'acceptance_rate', 'is_superhost', 'thumbnail_url', 'picture_url', 'neighborhood',
    'listings_count', 'total_listings_count', 'verifications', 'has_profile_pic',
    'identity_verified', 'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
])

# Users table
proc_users = pd.DataFrame(columns=['id', 'name'])

# Reviews table
proc_reviews = pd.DataFrame(columns=['id', 'listing_id', 'date', 'reviewer_id', 'comments'])

In [30]:
# Create a dictionary to map information from listings to the new dataframes
listings_info = {
    'proc_listings_description': {
        'listings_id': listings['id'],
        'description': listings['description'],
        'picture_url': listings['picture_url'],
        'accommodates': listings['accommodates'],
        'bathrooms': listings['bathrooms'],
        'bathrooms_text': listings['bathrooms_text'],
        'bedrooms': listings['bedrooms'],
        'beds': listings['beds'],
        'minimum_minimum_nights': listings['minimum_minimum_nights'],
        'maximum_minimum_nights': listings['maximum_minimum_nights'],
        'minimum_maximum_nights': listings['minimum_maximum_nights'],
        'maximum_maximum_nights': listings['maximum_maximum_nights'],
        'minimum_nights_avg_ntm': listings['minimum_nights_avg_ntm'],
        'maximum_nights_avg_ntm': listings['maximum_nights_avg_ntm']
    },
    'proc_listings': {
        'id': listings['id'],
        'name': listings['name'],
        'host_id': listings['host_id'],
        'neighbourhood_group': listings['neighbourhood_group_cleansed'],
        'neighbourhood': listings['neighbourhood_cleansed'],
        'city': listings['city'],
        'latitude': listings['latitude'],
        'longitude': listings['longitude'],
        'room_type': listings['room_type'],
        'property_type': listings['property_type'],
        'price': listings['price'],
        'minimum_nights': listings['minimum_nights'],
        'maximum_nights': listings['maximum_nights'],
        'license': listings['license']
    }, 
    'proc_listings_amenities': {
        'listing_id': listings['id'],
        'amenity': listings['amenities']
    }
}
# Map the dictionary to the new dataframes
for key, value in listings_info.items():
    for col, series in value.items():
        if key == 'proc_listings_description':
            proc_listings[col] = series
        elif key == 'proc_listings':
            proc_listings[col] = series
        elif key == 'proc_listings_amenities':
            proc_listings_amenities[col] = series

# Create a dictionary to map information from calendar to the new dataframes
calendar_info = {
    'proc_calendar': {
        'listing_id': calendar['listing_id'],
        'date': calendar['date'],
        'available': calendar['available'],
        'price': calendar['price'],
        'adjusted_price': calendar['adjusted_price'],
        'minimum_nights': calendar['minimum_nights'],
        'maximum_nights': calendar['maximum_nights']
    }
}
# Map the dictionary to the new dataframes
for key, value in calendar_info.items():
    for col, series in value.items():
        if key == 'proc_calendar':
            proc_calendar[col] = series

neighbourhoods_info = {
    'proc_neighborhoods': {
        'neighborhood_cleansed': neighbourhoods['neighbourhood'],
        'neighborhood_group': neighbourhoods['neighbourhood_group'],
        'city': neighbourhoods['city'],
    }
}

# Map the dictionary to the new dataframes
for key, value in neighbourhoods_info.items():
    for col, series in value.items():
        if key == 'proc_neighborhoods':
            proc_neighborhoods[col] = series
    

In [31]:
proc_cities['id'] = range(1, len(cities) + 1)
proc_cities['name'] = cities
proc_cities['country_code'] = 'ES'

In [32]:
proc_countries['iso_3166_1'] = 'ES'
proc_countries['name'] = 'Spain'

In [33]:
proc_neighborhood_groups['neighbourhood_group_cleansed'] = neighbourhoods['neighbourhood_group'].unique()
proc_neighborhood_groups['id'] = range(1, len(proc_neighborhood_groups) + 1)

In [34]:
proc_neighborhoods['id'] = range(1, len(neighbourhoods) + 1)
mapping = dict(proc_neighborhood_groups[['neighbourhood_group_cleansed', 'id']].values)
proc_neighborhoods['neighborhood_group'] = proc_neighborhoods['neighborhood_group'].map(mapping)

In [35]:
boolean_map = {'t': True, 'f': False}
proc_calendar['available'] = proc_calendar['available'].map(boolean_map)
proc_calendar['price'] = proc_calendar['price'].str.replace('$', '').str.replace(',', '').astype(float)
proc_calendar['adjusted_price'] = proc_calendar['adjusted_price'].str.replace('$', '').str.replace(',', '').astype(float)
proc_calendar['minimum_nights'] = proc_calendar['minimum_nights'].fillna(0).astype(int)
proc_calendar['maximum_nights'] = proc_calendar['maximum_nights'].fillna(0).astype(int)